<a href="https://colab.research.google.com/github/AlejandroDova/AlejandroDova/blob/master/Team_predict_WAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/content/data_games/fangraphs-leaderboards.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750 entries, 0 to 749
Data columns (total 23 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Season  750 non-null    int64  
 1   Team    750 non-null    object 
 2   TG      750 non-null    int64  
 3   G       750 non-null    int64  
 4   PA      750 non-null    int64  
 5   HR      750 non-null    int64  
 6   R       750 non-null    int64  
 7   RBI     750 non-null    int64  
 8   SB      750 non-null    int64  
 9   BB%     750 non-null    float64
 10  K%      750 non-null    float64
 11  ISO     750 non-null    float64
 12  BABIP   750 non-null    float64
 13  AVG     750 non-null    float64
 14  OBP     750 non-null    float64
 15  SLG     750 non-null    float64
 16  wOBA    750 non-null    float64
 17  xwOBA   0 non-null      float64
 18  wRC+    750 non-null    float64
 19  BsR     750 non-null    float64
 20  Off     750 non-null    float64
 21  Def     750 non-null    float64
 22  WA

In [7]:
START = 1999
END = 2023

In [24]:
def next_season(team):
    team = team.sort_values("Season")
    team["Next_WAR"] = team["WAR"].shift(-1)
    return team

df = df.groupby("team_code", group_keys=False).apply(next_season)

In [25]:
null_count = df.isnull().sum()

In [26]:
complete_cols = list(df.columns[null_count == 0])
df = df[complete_cols + ["Next_WAR"]].copy()

In [33]:
df

,Season,Team,TG,G,PA,HR,R,RBI,SB,BB%,...,SLG,wOBA,wRC+,BsR,Off,Def,WAR,team_code,Next_WAR,team_number
639,1999,ANA,0.990476,0.548023,0.907110,0.417969,0.622785,0.620370,0.306452,0.373469,...,0.359813,0.436060,0.155875,0.338435,-170.909616,0.715923,0.252116,0.0,0.715731,0
43,2000,ANA,0.990476,0.571751,0.960158,0.722656,0.816456,0.837302,0.424731,0.578207,...,0.821241,0.879628,0.651431,0.361594,49.934094,0.716134,0.715731,0.0,0.424728,0
387,2001,ANA,0.990476,0.563277,0.927801,0.417969,0.597468,0.605820,0.548387,0.305807,...,0.419340,0.454394,0.395328,0.434986,-57.472739,0.614543,0.424728,0.0,0.672858,0
76,2002,ANA,0.990476,0.836158,0.950033,0.394531,0.800000,0.802910,0.553763,0.198523,...,0.583553,0.656768,0.622679,0.619152,49.441254,0.635236,0.672858,0.0,0.477037,0
315,2003,ANA,0.990476,0.824294,0.904248,0.386719,0.654430,0.638889,0.618280,0.279442,...,0.465920,0.514035,0.465053,0.355746,-32.950143,0.630486,0.477037,0.0,0.545466,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,2019,WSN,0.990476,0.847458,0.936826,0.703125,0.827848,0.820106,0.548387,0.540616,...,0.714070,0.654198,0.603769,0.486701,34.699886,0.730996,0.652152,1.0,0.131034,33
723,2020,WSN,0.019048,0.024294,0.045565,0.058594,0.093671,0.099206,0.102151,0.428281,...,0.588843,0.588487,0.616706,0.375359,9.008268,0.454771,0.131034,1.0,0.483762,33
308,2021,WSN,0.990476,0.880791,0.902928,0.511719,0.639241,0.637566,0.225806,0.549914,...,0.491472,0.544084,0.574817,0.044936,-8.682225,0.565022,0.483762,1.0,0.190532,33
688,2022,WSN,0.990476,0.858757,0.877614,0.332031,0.486076,0.496032,0.327957,0.209903,...,0.255370,0.284821,0.414150,0.049301,-69.179060,0.313243,0.190532,1.0,0.231303,33


In [35]:
del df["xwOBA"]

KeyError: ignored

In [36]:
df["team_number"] = df["Team"].astype("category").cat.codes

In [37]:
df["team_number"]

639     0
43      0
387     0
76      0
315     0
       ..
93     33
723    33
308    33
688    33
658    33
Name: team_number, Length: 750, dtype: int8

In [38]:
df_full = df.copy()
df = df.dropna().copy()

In [39]:
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import TimeSeriesSplit

In [40]:
rr = Ridge(alpha=1)
split = TimeSeriesSplit(n_splits=3)
sfs = SequentialFeatureSelector(rr,
                                n_features_to_select=20,
                                direction="forward",
                                cv=split,
                                n_jobs=8
                               )

In [41]:
df

,Season,Team,TG,G,PA,HR,R,RBI,SB,BB%,...,SLG,wOBA,wRC+,BsR,Off,Def,WAR,team_code,Next_WAR,team_number
639,1999,ANA,0.990476,0.548023,0.907110,0.417969,0.622785,0.620370,0.306452,0.373469,...,0.359813,0.436060,0.155875,0.338435,-170.909616,0.715923,0.252116,0.0,0.715731,0
43,2000,ANA,0.990476,0.571751,0.960158,0.722656,0.816456,0.837302,0.424731,0.578207,...,0.821241,0.879628,0.651431,0.361594,49.934094,0.716134,0.715731,0.0,0.424728,0
387,2001,ANA,0.990476,0.563277,0.927801,0.417969,0.597468,0.605820,0.548387,0.305807,...,0.419340,0.454394,0.395328,0.434986,-57.472739,0.614543,0.424728,0.0,0.672858,0
76,2002,ANA,0.990476,0.836158,0.950033,0.394531,0.800000,0.802910,0.553763,0.198523,...,0.583553,0.656768,0.622679,0.619152,49.441254,0.635236,0.672858,0.0,0.477037,0
315,2003,ANA,0.990476,0.824294,0.904248,0.386719,0.654430,0.638889,0.618280,0.279442,...,0.465920,0.514035,0.465053,0.355746,-32.950143,0.630486,0.477037,0.0,0.545466,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2018,WSN,0.990476,0.911864,0.941448,0.546875,0.698734,0.705026,0.564516,0.662133,...,0.500048,0.537623,0.548557,0.557769,14.499740,0.606944,0.549446,1.0,0.652152,33
93,2019,WSN,0.990476,0.847458,0.936826,0.703125,0.827848,0.820106,0.548387,0.540616,...,0.714070,0.654198,0.603769,0.486701,34.699886,0.730996,0.652152,1.0,0.131034,33
723,2020,WSN,0.019048,0.024294,0.045565,0.058594,0.093671,0.099206,0.102151,0.428281,...,0.588843,0.588487,0.616706,0.375359,9.008268,0.454771,0.131034,1.0,0.483762,33
308,2021,WSN,0.990476,0.880791,0.902928,0.511719,0.639241,0.637566,0.225806,0.549914,...,0.491472,0.544084,0.574817,0.044936,-8.682225,0.565022,0.483762,1.0,0.190532,33


In [42]:
removed_columns = ["Next_WAR", "Team", "Off", "Season"]
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [43]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df.loc[:,selected_columns] = scaler.fit_transform(df[selected_columns])

In [44]:
df

,Season,Team,TG,G,PA,HR,R,RBI,SB,BB%,...,SLG,wOBA,wRC+,BsR,Off,Def,WAR,team_code,Next_WAR,team_number
639,1999,ANA,0.990476,0.548023,0.907110,0.417969,0.622785,0.620370,0.306452,0.373469,...,0.375509,0.436060,0.158017,0.338435,-170.909616,0.715923,0.252116,0.0,0.715731,0.0
43,2000,ANA,0.990476,0.571751,0.960158,0.722656,0.816456,0.837302,0.424731,0.578207,...,0.857068,0.879628,0.660386,0.361594,49.934094,0.716134,0.715731,0.0,0.424728,0.0
387,2001,ANA,0.990476,0.563277,0.927801,0.417969,0.597468,0.605820,0.548387,0.305807,...,0.437633,0.454394,0.400762,0.434986,-57.472739,0.614543,0.424728,0.0,0.672858,0.0
76,2002,ANA,0.990476,0.836158,0.950033,0.394531,0.800000,0.802910,0.553763,0.198523,...,0.609010,0.656768,0.631239,0.619152,49.441254,0.635236,0.672858,0.0,0.477037,0.0
315,2003,ANA,0.990476,0.824294,0.904248,0.386719,0.654430,0.638889,0.618280,0.279442,...,0.486246,0.514035,0.471446,0.355746,-32.950143,0.630486,0.477037,0.0,0.545466,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2018,WSN,0.990476,0.911864,0.941448,0.546875,0.698734,0.705026,0.564516,0.662133,...,0.521862,0.537623,0.556097,0.557769,14.499740,0.606944,0.549446,1.0,0.652152,1.0
93,2019,WSN,0.990476,0.847458,0.936826,0.703125,0.827848,0.820106,0.548387,0.540616,...,0.745221,0.654198,0.612069,0.486701,34.699886,0.730996,0.652152,1.0,0.131034,1.0
723,2020,WSN,0.019048,0.024294,0.045565,0.058594,0.093671,0.099206,0.102151,0.428281,...,0.614531,0.588487,0.625184,0.375359,9.008268,0.454771,0.131034,1.0,0.483762,1.0
308,2021,WSN,0.990476,0.880791,0.902928,0.511719,0.639241,0.637566,0.225806,0.549914,...,0.512912,0.544084,0.582718,0.044936,-8.682225,0.565022,0.483762,1.0,0.190532,1.0


In [45]:
sfs.fit(df[selected_columns], df["Next_WAR"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=Ridge(alpha=1), n_features_to_select=20,
                          n_jobs=8)

In [46]:
predictors = list(selected_columns[sfs.get_support()])

In [47]:
predictors

['TG',
 'G',
 'PA',
 'HR',
 'R',
 'RBI',
 'SB',
 'BB%',
 'K%',
 'ISO',
 'BABIP',
 'AVG',
 'OBP',
 'SLG',
 'wOBA',
 'wRC+',
 'BsR',
 'Def',
 'WAR',
 'team_code']

In [48]:
def backtest(data, model, predictors, start=5, step=1):
    all_predictions = []

    years = sorted(data["Season"].unique())

    for i in range(start, len(years), step):
        current_year = years[i]
        train = data[data["Season"] < current_year]
        test = data[data["Season"] == current_year]

        model.fit(train[predictors], train["Next_WAR"])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)
        combined = pd.concat([test["Next_WAR"], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [49]:
predictions = backtest(df, rr, predictors)

In [50]:
predictions

,actual,prediction
745,0.309580,0.232850
81,0.679735,0.527847
254,0.447541,0.426392
159,0.636341,0.455091
151,0.501304,0.490464
...,...,...
33,0.461469,0.603372
335,0.657200,0.467584
394,0.640922,0.423331
41,0.533029,0.565675


In [51]:
predictions.shape

(566, 2)

In [52]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predictions["actual"], predictions["prediction"])

0.024293161419438568

In [55]:
def player_history(df):
    df = df.sort_values("Season")

    df["player_season"] = range(0, df.shape[0])
    df["war_corr"] = list(df[["player_season", "WAR"]].expanding().corr().loc[(slice(None), "player_season"),"WAR"])
    df["war_corr"].fillna(0, inplace=True)

    df["war_diff"] = df["WAR"] / df["WAR"].shift(1)
    df["war_diff"].fillna(1, inplace=True)
    df["war_diff"][df["war_diff"] == np.inf] = 1

    return df

df = df.groupby("team_code", group_keys=False).apply(player_history)

In [59]:
df[['Team','Season','WAR','Next_WAR','war_diff']]

,Team,Season,WAR,Next_WAR,war_diff
639,ANA,1999,0.252116,0.715731,1.000000
43,ANA,2000,0.715731,0.424728,2.838893
387,ANA,2001,0.424728,0.672858,0.593418
76,ANA,2002,0.672858,0.477037,1.584211
315,ANA,2003,0.477037,0.545466,0.708971
...,...,...,...,...,...
203,WSN,2018,0.549446,0.652152,1.027251
93,WSN,2019,0.652152,0.131034,1.186926
723,WSN,2020,0.131034,0.483762,0.200926
308,WSN,2021,0.483762,0.190532,3.691872


In [60]:
def group_averages(df):
    return df["WAR"] / df["WAR"].mean()

In [62]:
df["war_season"] = df.groupby("Season", group_keys=False).apply(group_averages)

In [63]:
new_predictors = predictors + ["player_season", "war_corr", "war_season", "war_diff"]

In [65]:
predictions = backtest(df, rr, new_predictors)

In [66]:
mean_squared_error(predictions["actual"], predictions["prediction"])

0.02432366370150171

In [67]:
pd.Series(rr.coef_, index=new_predictors).sort_values()

RBI             -0.144769
R               -0.086516
OBP             -0.084900
BABIP           -0.073899
SLG             -0.071835
ISO             -0.056648
AVG             -0.055894
HR              -0.025328
K%              -0.015990
team_code       -0.010385
player_season   -0.002462
SB              -0.002389
war_diff         0.000500
Def              0.002700
war_corr         0.003677
G                0.007244
PA               0.022510
BsR              0.063943
TG               0.079139
WAR              0.086869
BB%              0.092635
wRC+             0.119116
war_season       0.177399
wOBA             0.188214
dtype: float64

In [68]:
diff = predictions["actual"] - predictions["prediction"]

In [69]:
merged = predictions.merge(df, left_index=True, right_index=True)

In [70]:
merged["diff"] = (predictions["actual"] - predictions["prediction"]).abs()

In [75]:
merged[["team_code", "Season", "Team", "WAR", "Next_WAR", "diff"]].sort_values(["diff"])

,team_code,Season,Team,WAR,Next_WAR,diff
631,0.787879,2013,SEA,0.255881,0.394162,0.000618
164,0.515152,2008,MIL,0.572511,0.572933,0.000890
168,0.060606,2008,ATL,0.568958,0.519957,0.002661
236,0.515152,2013,MIL,0.529429,0.488986,0.003052
13,0.909091,2009,TBR,0.798094,0.703602,0.003121
...,...,...,...,...,...,...
102,0.545455,2010,MIN,0.640095,0.132672,0.383522
208,0.515152,2019,MIL,0.547391,0.147264,0.398777
388,0.636364,2021,NYY,0.423466,0.827928,0.408663
93,1.000000,2019,WSN,0.652152,0.131034,0.426040


import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_0, *['team_code'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_1, *['Season'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_2, *['WAR'], **{})
chart

import numpy as np
from google.colab import autoviz

def value_plot(df, y, figscale=1):
  from matplotlib import pyplot as plt
  df[y].plot(kind='line', figsize=(8 * figscale, 4 * figscale), title=y)
  plt.gca().spines[['top', 'right']].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = value_plot(_df_3, *['Next_WAR'], **{})
chart

...

import numpy as np
from google.colab import autoviz

def scatter_plots(df, colname_pairs, figscale=1, alpha=.8):
  from matplotlib import pyplot as plt
  plt.figure(figsize=(len(colname_pairs) * 6 * figscale, 6 * figscale))
  for plot_i, (x_colname, y_colname) in enumerate(colname_pairs, start=1):
    ax = plt.subplot(1, len(colname_pairs), plot_i)
    df.plot(kind='scatter', x=x_colname, y=y_colname, s=(32 * figscale), alpha=alpha, ax=ax)
    ax.spines[['top', 'right',]].set_visible(False)
  plt.tight_layout()
  return autoviz.MplChart.from_current_mpl_state()

chart = scatter_plots(_df_8, *[[['team_code', 'Season'], ['Season', 'WAR'], ['WAR', 'Next_WAR'], ['Next_WAR', 'diff']]], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_9, *['diff', 'team_code', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_10, *['diff', 'Season', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_11, *['diff', 'WAR', None], **{})
chart

import numpy as np
from google.colab import autoviz

def time_series_multiline(df, timelike_colname, value_colname, series_colname, figscale=1, mpl_palette_name='Dark2'):
  from matplotlib import pyplot as plt
  import seaborn as sns
  figsize = (10 * figscale, 5.2 * figscale)
  palette = list(sns.palettes.mpl_palette(mpl_palette_name))
  def _plot_series(series, series_name, series_index=0):
    if value_colname == 'count()':
      counted = (series[timelike_colname]
                 .value_counts()
                 .reset_index(name='counts')
                 .rename({'index': timelike_colname}, axis=1)
                 .sort_values(timelike_colname, ascending=True))
      xs = counted[timelike_colname]
      ys = counted['counts']
    else:
      xs = series[timelike_colname]
      ys = series[value_colname]
    plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

  fig, ax = plt.subplots(figsize=figsize, layout='constrained')
  df = df.sort_values(timelike_colname, ascending=True)
  if series_colname:
    for i, (series_name, series) in enumerate(df.groupby(series_colname)):
      _plot_series(series, series_name, i)
    fig.legend(title=series_colname, bbox_to_anchor=(1, 1), loc='upper left')
  else:
    _plot_series(df, '')
  sns.despine(fig=fig, ax=ax)
  plt.xlabel(timelike_colname)
  plt.ylabel(value_colname)
  return autoviz.MplChart.from_current_mpl_state()

chart = time_series_multiline(_df_12, *['diff', 'Next_WAR', None], **{})
chart